<a href="https://colab.research.google.com/github/RK-05/Attendance-app/blob/main/Attendance_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***packages and imports***

In [ ]:
!pip install facenet-pytorch

     |████████████████████████████████| 1.9MB 8.1MB/s 


In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1

In [ ]:
mtcnn = MTCNN()
resnet = InceptionResnetV1(pretrained='vggface2').eval()

In [ ]:
import pandas as pd
from PIL import Image
import cv2

In [ ]:
from scipy.spatial.distance import cosine


In [ ]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok

In [ ]:
!pip install werkzeug 
from werkzeug.utils import secure_filename


In [ ]:
pip install flask_table

  Created wheel for flask-table: filename=Flask_Table-0.5.0-cp36-none-any.whl size=13959 sha256=a1ca6f9468d96fccb20e6c30b0fc105e231acf66058d19d76fc5a29361b24cea
  Stored in directory: /root/.cache/pip/wheels/09/bf/ed/f9217c8e82c1652c73420d49e6b2a42d404fe2fdb579adf948
Successfully built flask-table


In [ ]:
import os

In [ ]:
!pip install pytz

In [ ]:
!pip install flask_sqlalchemy

In [ ]:
from flask import Flask, flash, request, redirect, url_for,render_template,Response
from werkzeug.utils import secure_filename
from flask import send_from_directory
from scipy.spatial.distance import cosine
from flask_sqlalchemy import SQLAlchemy
from PIL import Image
from flask_table import Table, Col
from datetime import datetime
from tzlocal import get_localzone 
import pytz 
import numpy as np
from facenet_pytorch import MTCNN, extract_face
from numpy import asarray
import cv2
from __future__ import division
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json

In [ ]:
!pip install flask_wtf

     |████████████████████████████████| 174kB 8.9MB/s 


In [ ]:
from wtforms import Form, TextField, TextAreaField, validators, StringField, SubmitField
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField,SelectField
from wtforms.validators import DataRequired, Length, Email, EqualTo
from flask_wtf.file import FileField, FileAllowed, FileRequired

In [ ]:
import cv2
from tqdm import tqdm
import numpy as np
from PIL import Image

# ***Application***

In [ ]:
cd /content/drive/My Drive/Attendance

/content/drive/My Drive/Attendance


In [ ]:
dt=datetime.now(tz=pytz.timezone("Asia/Kolkata"))
dt

datetime.datetime(2021, 1, 4, 9, 54, 53, 923909, tzinfo=<DstTzInfo 'Asia/Kolkata' IST+5:30:00 STD>)

In [ ]:
from sqlalchemy import extract
UPLOAD_FOLDER = '/content/drive/My Drive/Attendance'
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif','mp4'}

app = Flask(__name__)
run_with_ngrok(app)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['SECRET_KEY'] = '7d441f27d441f27567d441f2b6176a'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////content/recorddb.db'
db = SQLAlchemy(app)

df = pd.DataFrame(columns=['Name','Value'])

json_file = open('emotion.json', 'r')
loaded_model_json = json_file.read()

json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("emotion.h5")
print("Loaded model from disk")
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

video_ls = []
vid=[]
emo=[]

class User(db.Model):
    emb = db.Column(db.String, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)


    def __repr__(self):
        return '<User %r>' % self.username

class Rec(db.Model):
    Id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    emotions = db.Column(db.String(80), nullable=False)
    date = db.Column(db.DateTime,nullable=False,default=datetime.now(tz=pytz.timezone("Asia/Kolkata")))

    def __repr__(self):
        return '<User %r>' % self.username

class Record(db.Model):
    Id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    date = db.Column(db.DateTime,nullable=False,default=datetime.now(tz=pytz.timezone("Asia/Kolkata")))

    def __repr__(self):
        return '<User %r>' % self.username

class Rec0(db.Model):
    Id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80),unique=True, nullable=False)
    emotions = db.Column(db.String(80), nullable=False)
    date = db.Column(db.DateTime,nullable=False,default=datetime.now(tz=pytz.timezone("Asia/Kolkata")))

    def __repr__(self):
        return '<User %r>' % self.username

db.create_all()

class Results(Table):
    Id = Col('Id')
    username = Col('username')
    emotions = Col('emotions')
    date = Col('date')

class Results_1(Table):
    id = Col('ID')
    name = Col('Name')
    description  = Col('Number of Days')

class Absentees(Table):
    id = Col('ID')
    name = Col('Name')

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

class LoginForm(FlaskForm):
    email = StringField('Email',
                        validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    remember = BooleanField('Remember Me')
    submit = SubmitField('Login')

class RegistrationForm(FlaskForm):
    username = StringField('Username',
                           validators=[DataRequired(), Length(min=2, max=20)])
    email = StringField('Email',
                        validators=[DataRequired()])
    gender = SelectField('Gender', [DataRequired()],
                        choices=[('Male', 'Male'),
                                 ('Female', 'Female'),
                                 ('Other', 'Other')])
    jobId = SelectField('Job Title', [DataRequired()],
                        choices=[('Farmer', 'farmer'),
                                 ('Corrupt Politician', 'politician'),
                                 ('No-nonsense City Cop', 'cop')])
    picture = FileField('Update Picture', validators=[FileAllowed(['jpg', 'png'])])
    submit = SubmitField('Submit')

choice=[]
choice.append(('All','All'))
for i in User.query.all():
  choice.append((i.username,i.username))
mon=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct',"Nov",'Dec']
month=[]
month.append(('All','All'))
for i in range(len(mon)):
  month.append((i+1,mon[i]))
class QueryForm(FlaskForm):
    username = SelectField('Username',[DataRequired()],
                           choices=choice)
    jobId = SelectField('Month',[DataRequired()],
                           choices=month)
    submit = SubmitField('Submit')


@app.route('/', methods=['GET', 'POST'])
def Attendence_upload():    
  return render_template('h1.html')

@app.route('/home', methods=['GET', 'POST'])
def home():    
  return render_template('hello.html')

@app.route('/login', methods=['GET', 'POST'])
def login1():
    error = None
    if request.method == 'POST':
        if request.form['username'] != 'admin' or request.form['password'] != 'admin':
            error = 'Invalid Credentials. Please try again.'
            flash('Login Unsuccessful. Please check username and password', 'danger')
        else:
            return redirect(url_for('home'))
    return render_template('login2.html', error=error)
'''
@app.route("/login", methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        if form.email.data == 'admin@cc.com' and form.password.data == 'password':
            flash('You have been logged in!', 'success')
            return redirect(url_for('ro'))
        else:
            flash('Login Unsuccessful. Please check username and password', 'danger')
    return render_template('login.html', title='Admin Login', form=form)'''

@app.route("/registration", methods=['GET', 'POST'])
def register():
    global df
    form = RegistrationForm()
    if form.validate_on_submit():
        flash(f'Acdiscription created for {form.username.data}!', 'success')
        if form.picture.data:
          f_name,f_ext = os.path.splitext(form.picture.data.filename)
          path = '/content/drive/My Drive/Attendance'+form.username.data+f_ext
          form.picture.data.save(path)
          img = Image.open(path)
          img = img.convert('RGB')
          img_cropped = mtcnn(img)
          img_embedding1 = resnet(img_cropped.unsqueeze(0))
          #mean_emb=(img_embedding1.detach().numpy()+img_embedding2.detach().numpy()+img_embedding3.detach().numpy())/3
          mean_emb = img_embedding1.detach().numpy()[0]
          if (form.username.data).lower() in df['Name'].unique():
            df = df.append({'Name':(form.username.data).lower(),'Value':mean_emb},ignore_index=True)
            embed=((df[df['Name']==(form.username.data).lower()]['Value'].mean()).tostring().hex())
            admin = User.query.filter_by(username=(form.username.data).lower()).update(dict(emb=embed))
            db.session.commit()
          else:
            df = df.append({'Name':(form.username.data).lower(),'Value':mean_emb},ignore_index=True)    
            guest = User(emb=mean_emb.tostring().hex(),username=(form.username.data).lower())
            db.session.add(guest)
            db.session.commit()
          return redirect(url_for('home'))
    return render_template('register.html', title='Register', form=form)


@app.route("/Record", methods=['GET', 'POST'])
def record():
    form = RegistrationForm()
    que=(db.session.query(User,db.func.count(Rec.username)).outerjoin(Rec,User.username == Rec.username).filter(extract('month', Rec.date) >= datetime.today().month,extract('year', Rec.date) >= datetime.today().year,extract('day', Rec.date) >= datetime.today().day).group_by(User.username).all())
    lis=[]
    for i in range(len(que)):
      lis.append(dict(id=i+1,name=que[i][0].username,description=que[i][1]))
    print(lis)
    table = Results_1(lis)
    table.border = True
    return render_template('results.html', table=table)

@app.route("/Records", methods=['GET', 'POST'])
def records():
    form = QueryForm()
    if request.method=='POST':
      if form.username.data != 'All' and form.jobId.data != 'All':
        que =db.session.query(Rec.username,db.func.count(Rec.username)).select_from(Rec).group_by(Rec.username).filter(Rec.username==form.username.data,extract('month', Rec.date) == form.jobId.data).all()
        lis=[]
        for i in range(len(que)):
          lis.append(dict(id=i+1,name=que[i][0],description=que[i][1]))
        table = Results_1(lis)
        table.border = True
        return render_template('results.html', table=table)
      elif form.username.data == 'All' and form.jobId.data != 'All':
        que=db.session.query(Rec.username,db.func.count(Rec.username)).select_from(Rec).group_by(Rec.username).filter(extract('month', Rec.date) == form.jobId.data).order_by(db.func.count(Rec.username).desc()).all()
        bar_labels=[]
        bar_values=[]
        lis=[]
        for i in range(len(que)):
          bar_labels.append((que[i][0]))
          bar_values.append((que[i][1]))
          lis.append(dict(id=i+1,name=que[i][0],description=que[i][1]))
        table = Results_1(lis)
        table.border = True 
        return render_template('bar_chart.html',table=table,title='Monthly Attendance', max=20, labels=bar_labels, values=bar_values)
      elif form.username.data != 'All' and form.jobId.data == 'All':
        que=(db.session.query(User,db.func.count(Rec.username),extract('month', Rec.date)).outerjoin(Rec,User.username == Rec.username).filter(Rec.username=='rohan').group_by(extract('month', Rec.date)).all())
        bar_labels=[]
        bar_values=[]
        lis=[]
        for i in range(len(que)):
          for j in month[1:]:
            if j[0] == que[i][2]:
              bar_labels.append((j[1]))
              bar_values.append((que[i][1]))
              lis.append(dict(id=i+1,name=j[1],description=que[i][1]))
            else:
              bar_labels.append((j[1]))
              bar_values.append((0))
        table = Results_1(lis)
        table.border = True 
        return render_template('bar_chart.html',table=table,title=form.username.data+' '+'Monthly Attendance', max=20, labels=bar_labels, values=bar_values)  
      elif form.username.data == 'All' and form.jobId.data == 'All':
        que=db.session.query(Rec.username,db.func.count(Rec.username)).select_from(Rec).group_by(Rec.username).filter(extract('year', Rec.date) ==2020 ).order_by(db.func.count(Rec.username).desc()).all()
        bar_labels=[]
        bar_values=[]
        lis=[]
        for i in range(len(que)):
          bar_labels.append((que[i][0]))
          bar_values.append((que[i][1]))
          lis.append(dict(id=i+1,name=que[i][0],description=que[i][1]))
        table = Results_1(lis)
        table.border = True 
        return render_template('bar_chart.html',table=table,title='Monthly Attendance', max=20, labels=bar_labels, values=bar_values)
    return render_template('Record.html', form=form)



@app.route('/mobile_upload', methods=['GET', 'POST'])
def upload_mobile():
  return render_template('mobile.html')


@app.route('/mobile_video_upload', methods=['GET', 'POST'])
def mobile_video_upload():
    global file,name,fin,emotion
    a=0
    mob_name=[]
    mob_emotion=[]
    Rec0.query.delete()
    db.session.commit()
    cont=User.query.all()
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
          path = '/content/drive/My Drive/Attendance'+file.filename
          file.save(path)
          v_cap = cv2.VideoCapture(path)
          v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
          # Loop through video
          batch_size = 16
          frames = []
          faces = []
          video_ls=[]
          cont=User.query.all()
          detector = MTCNN(min_face_size=200)
          # detect faces in the image
          for i in (range(v_len)):
              # Load frame
              success, frame = v_cap.read()
              if not success:
                  continue 
              # Add to batch
              img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
              # When batch is full, detect faces and reset batch list
              results = detector.detect(img)
              if len(results)>0:
                        for i in range(len(results[0])):
                          extract_face(img, results[0][i], save_path='group/detected_face_stu_vid_{}.png'.format(i))
                          full_size_image = cv2.imread('group/detected_face_stu_vid_{}.png'.format(i))
                          gray=cv2.cvtColor(full_size_image,cv2.COLOR_BGR2GRAY)
                          #detecting faces
                          for (x, y, w, h) in  results[0]:
                                  roi_gray = gray[int(y):int(h), int(x):int(w)]
                                  cropped_img = np.expand_dims(np.expand_dims(cv2.resize(gray, (48, 48)), -1), 0)
                                  cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
                                  #predicting the emotion
                                  yhat= loaded_model.predict(cropped_img)
                          #emotion.append(labels[int(np.argmax(yhat))])
                          fi = Image.open('group/detected_face_stu_vid_{}.png'.format(i))
                          try:
                            img_cropped = mtcnn(fi)
                            img_embedding = resnet(img_cropped.unsqueeze(0)).detach().numpy()[0]
                            for j in cont:
                              vec=(np.frombuffer(bytes.fromhex(j.emb),dtype=np.float32))
                              if cosine(img_embedding, vec)<0.45:
                                video_ls.append((j.username,labels[int(np.argmax(yhat))],cosine(img_embedding, vec),i))
                                a+=1
                          except:
                              pass
          if a>0:
            video_ls.sort(key = lambda x:x[2])
            for i in video_ls:
              if i[0] not in mob_name:
                mob_name.append(i[0])
                mob_emotion.append(i[1])
                for i in cont:
                  if i.username in mob_name:
                    guest = Rec0(username=i.username,emotions=mob_emotion[mob_name.index(i.username)],date=datetime.now(tz=pytz.timezone("Asia/Kolkata")))
                    guest_t = Rec(username=i.username,emotions=mob_emotion[mob_name.index(i.username)],date=datetime.now(tz=pytz.timezone("Asia/Kolkata")))
                    db.session.add(guest)
                    db.session.add(guest_t)
                    db.session.commit()
                table1 = Results(Rec0.query.all())
                table1.border = True
                lis=[]
                Qr = [i.username for i in User.query.all() if i.username not in [i.username for i in Rec0.query.filter(extract('month', Rec0.date) >= datetime.today().month,extract('year', Rec0.date) >= datetime.today().year,extract('day', Rec0.date) >= datetime.today().day).all()]]
                for i in range(len(Qr)):
                  lis.append(dict(id=i+1,name=Qr[i]))
                table = Absentees(lis)
                table.border = True
                return render_template('result_a.html', table=table1,table1=table)
              else:
                print('no match')
                return " no match"
    return  render_template('upload.html')


@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'],
                               filename)

@app.route('/live_upload')
def live_upload():
    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>Title</title>
        <style>
          html {
                  overflow: hidden;
                  height: 100%;
                }

                video {
                  width: 90%;
                  height: 60%;
                  position: absolute;
                  display: block;
                  top: 50px;
                  left: 50px;
                  object-fit: cover;
                }

                body {
                  background-image: url('/static/c&c-back.png');
                  background-repeat: no-repeat;
                  background-attachment: fixed;  
                  background-size: cover;
                  margin: 0;
                  height: 100%;
                  width: 100%;
                  position: fixed;
                  top: 0;
                  left: 0;
                  bottom: 0;
                  right: 0;
                }
        </style>      
    </head>
    <body>
    <video id="video" width="640" height="480" autoplay></video>
    <button id="snap">Snap Photo</button>
    <button id="stop" title="stop" onclick="stopvideo()">stop</button>
    <button id="done" title="done" onclick="myFunction()">done</button>
    <canvas id="canvas" width="640" height="480"></canvas>
    </body>
    <script>

    var video = document.getElementById('video');
    if(navigator.mediaDevices && navigator.mediaDevices.getUserMedia) {
        navigator.mediaDevices.getUserMedia({ video: true }).then(function(stream) {
            //video.src = window.URL.createObjectURL(stream);
            video.srcObject = stream;
            video.play();
        });
    }

    var canvas = document.getElementById('canvas');
    var context = canvas.getContext('2d');
    var video = document.getElementById('video');
    function myFunction() {
      location.replace("/results")
    };
    var content = '<a id="a"><b id="b">hey!</b></a>';
    var blob = new Blob([content], { type: "text/xml"});
    function stopvideo(){
      video.pause()
      var formData = new FormData()
      formData.append('webmasterfile', blob);
      var request = new XMLHttpRequest();
      request.open('POST', '/video',true);
      request.send(formData);
    };
    function grab(){
      context.drawImage(video, 0, 0, 640, 480);
      canvas.toBlob(upload,'image/jpeg');
    };

    // Trigger photo take
    document.getElementById("snap").addEventListener("click", function() {
        context.drawImage(video, 0, 0, 640, 480);
    canvas.toBlob(upload,'image/jpeg');
    function upload(blob){
      var fd = new FormData();
      fd.append('filetoupload',blob);
      var request = new XMLHttpRequest();
      request.open('POST', '/video', true);
      request.onload = function(){
        context.drawImage(video, 0, 0, 640, 480);
        canvas.toBlob(upload,'image/jpeg');
      };
      request.send(fd);
    }
    });


</script>
</html>
    """

@app.route('/Attendence_upload', methods=['GET', 'POST'])
def Attendencee_upload():
    global file,name,fin,emotion
    a=0
    name=[]
    fin=[]
    emotion=[]
    Rec0.query.delete()
    db.session.commit()
    cont=User.query.all()
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            #mtcnn = MTCNN(keep_all=True,select_largest=False)
            img = Image.open(file)
            img = img.convert('RGB')
            results = mtcnn.detect(img)
            for i in range(len(results[0])):
              extract_face(img, results[0][i], save_path='group/detected_face_stu_{}.png'.format(i))
              full_size_image = cv2.imread('group/detected_face_stu_{}.png'.format(i))
              gray=cv2.cvtColor(full_size_image,cv2.COLOR_BGR2GRAY)
              #detecting faces
              for (x, y, w, h) in  results[0]:
                      roi_gray = gray[int(y):int(h), int(x):int(w)]
                      cropped_img = np.expand_dims(np.expand_dims(cv2.resize(gray, (48, 48)), -1), 0)
                      cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
                      #predicting the emotion
                      yhat= loaded_model.predict(cropped_img)
              #emotion.append(labels[int(np.argmax(yhat))])
              fi = Image.open('group/detected_face_stu_{}.png'.format(i))
              try:
                img_cropped = mtcnn(fi)
                img_embedding = resnet(img_cropped.unsqueeze(0)).detach().numpy()[0]
                for j in cont:
                  vec=(np.frombuffer(bytes.fromhex(j.emb),dtype=np.float32))
                  if cosine(img_embedding, vec)<0.45:
                    name.append((j.username,labels[int(np.argmax(yhat))],cosine(img_embedding, vec),i))
                    a+=1
              except:
                print('could not detect',i)
                pass
            if a>0:
              name.sort(key = lambda x:x[2])
              for i in name:
                if i[0] not in fin:
                  fin.append(i[0])
                  emotion.append(i[1])
              print({'names':name,'emotion':fin})
              for i in cont:
                if i.username in fin:
                  guest = Rec0(username=i.username,emotions=emotion[fin.index(i.username)],date=datetime.now(tz=pytz.timezone("Asia/Kolkata")))
                  guest_t = Rec(username=i.username,emotions=emotion[fin.index(i.username)],date=datetime.now(tz=pytz.timezone("Asia/Kolkata")))
                  db.session.add(guest)
                  db.session.add(guest_t)
                  db.session.commit()
              table1 = Results(Rec0.query.all())
              table1.border = True
              lis=[]
              Qr = [i.username for i in User.query.all() if i.username not in [i.username for i in Rec0.query.filter(extract('month', Rec0.date) >= datetime.today().month,extract('year', Rec0.date) >= datetime.today().year,extract('day', Rec0.date) >= datetime.today().day).all()]]
              for i in range(len(Qr)):
                lis.append(dict(id=i+1,name=Qr[i]))
              table = Absentees(lis)
              table.border = True
              return render_template('result_a.html', table=table1,table1=table)
            else:
              print('no match')
              return " no match"
    return  render_template('upload.html')

@app.route('/video', methods=['POST'])
def video():
      global file1,video_ls,vid,emo
      file1 = request.files
      Rec0.query.delete()
      db.session.commit()
      cont = User.query.all()
      if request.method=='POST':
          try:
              print('entered')
              #chk_fl=request.file['filetoupload']
              img = Image.open(file1['filetoupload'])
              img = img.convert('RGB')
              results = mtcnn.detect(img)
              for i in range(len(results[0])):
                extract_face(img, results[0][i], save_path='group/detected_face_stu_vid_{}.png'.format(i))
                full_size_image = cv2.imread('group/detected_face_stu_vid_{}.png'.format(i))
                gray=cv2.cvtColor(full_size_image,cv2.COLOR_BGR2GRAY)
                #detecting faces
                for (x, y, w, h) in  results[0]:
                        roi_gray = gray[int(y):int(h), int(x):int(w)]
                        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(gray, (48, 48)), -1), 0)
                        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
                        #predicting the emotion
                        yhat= loaded_model.predict(cropped_img)
                #emotion.append(labels[int(np.argmax(yhat))])
                fi = Image.open('group/detected_face_stu_vid_{}.png'.format(i))
                try:
                  img_cropped = mtcnn(fi)
                  img_embedding = resnet(img_cropped.unsqueeze(0)).detach().numpy()[0]
                  for j in cont:
                    vec=(np.frombuffer(bytes.fromhex(j.emb),dtype=np.float32))
                    if cosine(img_embedding, vec)<0.45:
                      video_ls.append((j.username,labels[int(np.argmax(yhat))],cosine(img_embedding, vec),i))
                      a+=1
                except:
                    pass
                return "recorded"
          except:
            if file1['webmasterfile']:
              print('entered_2')
              #chk_fl=request.file['webmasterfile']
              if len(video_ls)>0:
                video_ls.sort(key = lambda x:x[2])
                for i in video_ls:
                  if i[0] not in vid:
                    vid.append(i[0])
                    emo.append(i[1])
                print(video_ls)
                print(vid)
                for i in cont:
                  if i.username in vid:
                    guest = Rec0(username=i.username,emotions=emo[vid.index(i.username)])
                    guest_t = Rec(username=i.username,emotions=emo[vid.index(i.username)])
                    db.session.add(guest)
                    db.session.add(guest_t)
                    db.session.commit()
                    print('match')
                return "match"
              else:
                print('no match')
                return " no match"
      else:
          return "Invalid request"

@app.route('/results')
def results():
    video_ls=[]
    vid=[]
    emo=[]
    Recs = Rec.query.filter(extract('month', Rec.date) >= datetime.today().month,extract('year', Rec.date) >= datetime.today().year,extract('day', Rec.date) >= datetime.today().day).all()
    table1 = Results(Recs)
    table1.border = True
    lis=[]
    Qr = [i.username for i in User.query.all() if i.username not in [i.username for i in Rec.query.filter(extract('month', Rec.date) >= datetime.today().month,extract('year', Rec.date) >= datetime.today().year,extract('day', Rec.date) >= datetime.today().day).all()]]
    for i in range(len(Qr)):
      lis.append(dict(id=i+1,name=Qr[i]))
    table = Absentees(lis)
    table.border = True
    return render_template('result_a.html', table=table1,table1=table)

@app.route('/')
def download():
	return render_template('download.html')

@app.route('/download/report/csv')
def download_report():
  try:
    result = User.query.all()
    output = io.StringIO()
    writer = csv.writer(output)
    line = ['username','emb']
    writer.writerow(line)
    for row in result:
      line = [row.username,row.emb]
      writer.writerow(line)
    output.seek(0)
    return Response(output, mimetype="text/csv", headers={"Content-Disposition": "attachment;filename=employee_report.csv"})
  except Exception as e:
      print(e)

if __name__ == '__main__':
   app.run()

Loaded model from disk
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production


/usr/local/lib/python3.6/dist-packages/flask_sqlalchemy/__init__.py:834: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1a6d3add240b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Jan/2021 04:56:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2021 04:56:50] "GET /static/c%26c-back.png HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2021 04:56:52] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Jan/2021 04:56:55] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2021 04:56:55] "GET /static/adminlogo.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2021 04:57:06] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [04/Jan/2021 04:57:07] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2021 04:57:11] "GET /Attendence_upload HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2021 04:57:11] "GET /static/uploadimg.png HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2021 04:57:22] "GET /login HTTP/1.1" 200 -


# ***Trial codes***

In [ ]:
    f1='aditya'
    f2="All"
    if f1 != 'All' and f2 != 'All':
      que =db.session.query(Rec.username,db.func.count(Rec.username)).select_from(Rec).group_by(Rec.username).filter(Rec.username==f1,extract('month', Rec.date) == f2).all()
    elif f1 == 'All' and f2 != 'All':
      que=db.session.query(Rec.username,db.func.count(Rec.username)).select_from(Rec).group_by(Rec.username).filter(extract('month', Rec.date) == f2).order_by(db.func.count(Rec.username).desc()).all()
    elif f1 != 'All' and f2 == 'All':
      que=(db.session.query(User,db.func.count(Rec.username),extract('month', Rec.date)).outerjoin(Rec,User.username == Rec.username).filter(Rec.username==f1).group_by(extract('month', Rec.date)).all())
    elif f1 == 'All' and f2 == 'All':
      que=db.session.query(Rec.username,db.func.count(Rec.username)).select_from(Rec).group_by(Rec.username).filter(extract('year', Rec.date) ==2020 ).order_by(db.func.count(Rec.username).desc()).all()
    print(que)
    lis=[]
    for i in range(len(que)):
      lis.append(dict(id=i+1,name=que[i][0].username,description=que[i][1]))
    print(lis)

[(<User 'aditya'>, 7, 8)]
[{'id': 1, 'name': 'aditya', 'description': 7}]


In [ ]:
month

[('All', 'All'),
 (1, 'Jan'),
 (2, 'Feb'),
 (3, 'Mar'),
 (4, 'Apr'),
 (5, 'May'),
 (6, 'Jun'),
 (7, 'Jul'),
 (8, 'Aug'),
 (9, 'Sep'),
 (10, 'Oct'),
 (11, 'Nov'),
 (12, 'Dec')]

In [ ]:
        que=(db.session.query(User,db.func.count(Rec.username),extract('month', Rec.date)).outerjoin(Rec,User.username == Rec.username).filter(Rec.username=='rohan').group_by(extract('month', Rec.date)).all())
        bar_labels=[]
        bar_values=[]
        lis=[]
        for i in range(len(que)):
          for j in month[1:]:
            if j[0] == que[i][2]:
              bar_labels.append((j[1]))
              bar_values.append((que[i][1]))
              lis.append(dict(id=i+1,name=j[1],description=que[i][1]))
            else:
              bar_labels.append((j[1]))
              bar_values.append((0)) 
        lis

[{'description': 5, 'id': 1, 'name': 'Aug'}]

In [ ]:
8 in que[0]

True

In [ ]:
[i.username for i in User.query.all() if i.username not in [i.username for i in Rec.query.filter(extract('month', Rec.date) >= datetime.today().month,extract('year', Rec.date) >= datetime.today().year,extract('day', Rec.date) >= datetime.today().day).all()]]

['lebron']

In [ ]:
lis=[]
for i in range(len(Qr)):
  lis.append(dict(id=i+1,name=Qr[i][0].username,description=Qr[i][1]))
print(lis)

[{'id': 1, 'name': 'lebron'}]

In [ ]:
(db.session.query(User,db.func.count(Rec.username),extract('month', Rec.date)).join(Rec,User.username == Rec.username).filter(extract('month', Rec.date)==8).group_by(Rec.username=='aditya').all())


AttributeError: ignored

In [ ]:
db.session.query(Rec.username,db.func.count(Rec.username)).select_from(Rec).group_by(Rec.username).filter(extract('year', Rec.date) == 2020).all()

[('aditya', 8),
 ('chinmay', 8),
 ('mihir', 12),
 ('nikhil', 6),
 ('rk', 4),
 ('rohan', 8),
 ('shantanu', 12),
 ('shreyas', 6),
 ('vadi', 8)]

In [ ]:
UPLOAD_FOLDER = '/content/uploads'
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'}

app = Flask(__name__)
run_with_ngrok(app)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config.from_object(__name__)
app.config['SECRET_KEY'] = '7d441f27d441f27567d441f2b6176a'

@app.route('/login', methods=['GET', 'POST'])
def login1():
    error = None
    if request.method == 'POST':
        if request.form['username'] != 'admin' or request.form['password'] != 'admin':
            error = 'Invalid Credentials. Please try again.'
            flash('Login Unsuccessful. Please check username and password', 'danger')
        else:
            return redirect(url_for('home'))
    return render_template('login2.html', error=error)

@app.route('/', methods=['GET', 'POST'])
def Attendence_upload():   
      if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename)) 
            return redirect(url_for('home'))
      return render_template('upload.html')

@app.route('/home', methods=['GET', 'POST'])
def home():    
  return render_template('hello.html')

 
@app.route("/registration", methods=['GET', 'POST'])
def register():
    form = RegistrationForm()
    if form.validate_on_submit():
        flash(f'Account created for {form.username.data}!', 'success')
        if form.picture.data:
          print((form.username.data).lower())
          f_name,f_ext = os.path.splitext(form.picture.data.filename)
          path = '/content/static/uploads/'+form.username.data+f_ext
          form.picture.data.save(path)
        return redirect(url_for('home'))
    return render_template('register.html', title='Register', form=form)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c5f3fe447b9e.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Aug/2020 12:19:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 12:19:28] "GET /static/c%26c-back.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Aug/2020 12:19:30] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
app = Flask(__name__)
run_with_ngrok(app)
app.config['SECRET_KEY'] = '7d441f27d441f27567d441f2'

@app.route('/', methods=['GET', 'POST'])
def Attended():    
  return render_template('upload.html')

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 1182, in run
    self.function(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.6/dist-packages/flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
  File "/usr/local/lib/python3.6/dist-packages/flask_ngrok.py", line 38, in _run_ngrok
    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
IndexError: list index out of range



In [ ]:
pip install flask flask-uploads flask-dropzone

  Created wheel for flask-uploads: filename=Flask_Uploads-0.2.1-cp36-none-any.whl size=8068 sha256=7234bf71daad96d1bfbafa13898e190c4890d4067f5627ab8685ce9d1cf52916
  Stored in directory: /root/.cache/pip/wheels/af/41/f6/0c357a43a5680e7690ce50337ab7fe9304795910ac10e15ce6
Successfully built flask-uploads


In [ ]:
  
from flask import Flask, redirect, render_template, request, session, url_for
from flask_dropzone import Dropzone
from flask_uploads import UploadSet, configure_uploads, IMAGES, patch_request_class

import os

app = Flask(__name__)
#run_with_ngrok(app)
dropzone = Dropzone(app)


app.config['SECRET_KEY'] = 'supersecretkeygoeshere'

# Dropzone settings
app.config['DROPZONE_UPLOAD_MULTIPLE'] = True
app.config['DROPZONE_ALLOWED_FILE_CUSTOM'] = True
app.config['DROPZONE_ALLOWED_FILE_TYPE'] = 'image/*'
app.config['DROPZONE_REDIRECT_VIEW'] = 'results'

# Uploads settings
app.config['UPLOADED_PHOTOS_DEST'] = os.getcwd() + '/uploads'

photos = UploadSet('photos', IMAGES)
configure_uploads(app, photos)
patch_request_class(app)  # set maximum file size, default is 16MB


@app.route('/', methods=['GET', 'POST'])
def index():
    
    # set session for image results
    if "file_urls" not in session:
        session['file_urls'] = []
    # list to hold our uploaded image urls
    file_urls = session['file_urls']

    # handle image upload from Dropszone
    if request.method == 'POST':
        file_obj = request.files
        for f in file_obj:
            file = request.files.get(f)
            
            # save the file with to our photos folder
            filename = photos.save(
                file,
                name=file.filename    
            )

            # append image urls
            file_urls.append(photos.url(filename))
            
        session['file_urls'] = file_urls
        return "uploading..."
    # return dropzone template on GET request    
    return render_template('index.html')


@app.route('/results')
def results():
    
    # redirect to home if no images to display
    if "file_urls" not in session or session['file_urls'] == []:
        return redirect(url_for('index'))
        
    # set the file_urls and remove the session variable
    file_urls = session['file_urls']
    session.pop('file_urls', None)
    
    return render_template('results.html', file_urls=file_urls)

ImportError: ignored

In [ ]:
ALLOWED_EXTENSIONS = ['mp']
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
filename=('rk.mp')
filename.rsplit('.', 1)[1].lower()

'mp'

In [ ]:
cd /content/

/content


In [ ]:

path='/content/Attendancevideo_20200825_181616.mp4'
v_cap = cv2.VideoCapture('Attendancevideo_20200825_181616.mp4')
v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
a=0
# Loop through video
batch_size = 16
frames = []
faces = []
video_ls=[]
cont=User.query.all()
detector = MTCNN(min_face_size=200)
# detect faces in the image
for i in tqdm(range(v_len)):
    
    # Load frame
    success, frame = v_cap.read()
    if not success:
        continue
        
    # Add to batch
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # When batch is full, detect faces and reset batch list
    results = detector.detect(img)
    if len(results)>0:
              for i in range(len(results[0])):
                extract_face(img, results[0][i], save_path='group/detected_face_stu_vid_{}.png'.format(i))
                full_size_image = cv2.imread('group/detected_face_stu_vid_{}.png'.format(i))
                gray=cv2.cvtColor(full_size_image,cv2.COLOR_BGR2GRAY)
                #detecting faces
                for (x, y, w, h) in  results[0]:
                        roi_gray = gray[int(y):int(h), int(x):int(w)]
                        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(gray, (48, 48)), -1), 0)
                        cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
                        #predicting the emotion
                        yhat= loaded_model.predict(cropped_img)
                #emotion.append(labels[int(np.argmax(yhat))])
                fi = Image.open('group/detected_face_stu_vid_{}.png'.format(i))
                try:
                  img_cropped = mtcnn(fi)
                  img_embedding = resnet(img_cropped.unsqueeze(0)).detach().numpy()[0]
                  for j in cont:
                    vec=(np.frombuffer(bytes.fromhex(j.emb),dtype=np.float32))
                    if cosine(img_embedding, vec)<0.45:
                      video_ls.append((j.username,labels[int(np.argmax(yhat))],cosine(img_embedding, vec),i))
                      a+=1
                except:
                    pass
if a>0:
  mob_name=[]
  mob_emotion=[]
  video_ls.sort(key = lambda x:x[2])
  for i in video_ls:
    if i[0] not in mob_name:
      mob_name.append(i[0])
      mob_emotion.append(i[1])
  print({'names':mob_name,'emotion':mob_emotion})




  0%|          | 0/46 [00:00<?, ?it/s]


TypeError: ignored

In [ ]:
video_ls

[('rohan', 'Neutral', 0.24226510524749756, 0),
 ('rohan', 'Happy', 0.2348039150238037, 0),
 ('rohan', 'Neutral', 0.25592881441116333, 0),
 ('rohan', 'Neutral', 0.23438429832458496, 0),
 ('rohan', 'Neutral', 0.2427973747253418, 0),
 ('rohan', 'Neutral', 0.23211044073104858, 0),
 ('rohan', 'Neutral', 0.2277495265007019, 0),
 ('rohan', 'Neutral', 0.23058819770812988, 0),
 ('rohan', 'Neutral', 0.22630006074905396, 0),
 ('rohan', 'Neutral', 0.2592211365699768, 0),
 ('rohan', 'Neutral', 0.2226090431213379, 0),
 ('rohan', 'Neutral', 0.24656546115875244, 0),
 ('rohan', 'Neutral', 0.24557149410247803, 0),
 ('rohan', 'Neutral', 0.23086875677108765, 0),
 ('rohan', 'Neutral', 0.2392643690109253, 0),
 ('rohan', 'Neutral', 0.23021602630615234, 0),
 ('rohan', 'Neutral', 0.2549723982810974, 0),
 ('rohan', 'Surprise', 0.25198453664779663, 0),
 ('rohan', 'Happy', 0.2202933430671692, 0),
 ('rohan', 'Neutral', 0.2261507511138916, 0),
 ('rohan', 'Happy', 0.2228265404701233, 0),
 ('rohan', 'Neutral', 0.2380

In [ ]:
mob_name=[]
mob_emotion=[]
video_ls.sort(key = lambda x:x[2])
for i in video_ls:
  if i[0] not in mob_name:
    mob_name.append(i[0])
    mob_emotion.append(i[1])
print({'names':mob_name,'emotion':mob_emotion})

{'names': ['rohan'], 'emotion': ['Happy']}


In [ ]:
<meta name="viewport" content="width=device-width, initial-scale=1">
<meta name="viewport" content="width=device-width, initial-scale=1.0">


In [ ]:
@app.route('/upload', methods=['GET', 'POST'])
def Attendencee_upload():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
          #--your-code--
        else:
          #--
      else:
        #--
    return  render_template('upload.html')

In [ ]:
name='rohan'
print(name[::-1])

nahor


In [ ]:
x=0
lis=[1,2,3,4,5,3,4,2,4,6,4,7,4,8,11,2,3,4,3]
for i in lis:
  if x<i:
    x=i
print(x)

11
